In [ ]:
import numpy as np
import os
import os.path
from pathlib import Path
import time
import shutil
import csv
import math
import glob
from ARMA3_DG_BB import get_BB_BackroundSubtraction
from ARMA3_DG_CLASSES import ARMA3_DG_vehiclesClasses
import cv2

In [4]:
def move_file(src, dest):
    """ 
    Given a file 'src', move that file to the correct file specified by 'dest' 
    """
    # exit with error code 4 if this fails...
    img_moved = False
    while not img_moved:
        try:
            shutil.move(src, dest)
        except OSError as e:
            if e.errno == 13:
                # print('File move error caught with OSError.errno 13') #  for debugging
                # This sleep could be 0.2 secs but I don't want to use this high a sleep time.
                # Could be an issue if the user makes the camera rotation time too low
                time.sleep(0.1)
                continue
            # raise the exception if it is not this particular error
            print('ERROR:')
            print('The image file mover process has encountered some unknown error. Please restart the script.')
            print('See the python error description here:')
            print(e)
            exit(4)
        # explicitly raise any other exception type
        except Exception as weirdE:
            print('ERROR:')
            print('The image file mover process has encountered some unknown error. Please restart the script.')
            print('See the python error description here:')
            print(weirdE)
            exit(4)
        img_moved = True

def get_next_id(directory):
    """
    Get a folder, search for all files with integet as their name.
    Return the next integer in line.
    """
    existing_ids = []
    
    # Scan the directory for image files
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):  # You can add more extensions if needed
            try:
                # Extract the numeric ID from the filename (e.g., "1.jpg", "25.png")
                id_part = filename.split('.')[0]
                existing_ids.append(int(id_part))
            except ValueError:
                continue  # Skip if the filename doesn't have a valid integer ID
    
    # If there are existing IDs, get the next one
    next_id = max(existing_ids, default=0) + 1
    return next_id



In [9]:
import glob

screenshot_path = Path(os.path.expanduser('~\\Documents\\Arma 3\\Screenshots'))
dest_dir        = Path(os.path.expanduser('D:\\synthetic_dataset\\arma'))

def SearchAndMove_Empty(screenshot_path,dest_dir):
    # Using '*' pattern 
    for filepath in glob.glob(str(screenshot_path)+'/*empty.png'):
        # Extract information from filename
        file_name = filepath.split('\\')[-1]
        print("SearchAndMove_Empty::Parsing"+file_name)
        filename_tokens = file_name.split('-')
        world   = filename_tokens[0]
        c_x,c_y = [float(coo) for coo in filename_tokens[1][1:-1].split(',') ]
        time    = int(filename_tokens[2])
        overcast= float(filename_tokens[3])
        rain    = float(filename_tokens[4])
        fog     = float(filename_tokens[5])

        # Move to world directory
        dest_dir_path = dest_dir / Path(world+'_empty')
        if not Path.exists(dest_dir_path):
            os.makedirs(str(dest_dir_path), 0o777)  # create the directory if it doesnt exist
        print("SearchAndMove_Empty::DestinationDir::"+str(dest_dir_path))
        # Get Ascending ID
        ID = get_next_id(dest_dir_path)

        # Find or create Annotation File
        annotation_file = dest_dir_path / Path('AnnotationData.csv')
        if not annotation_file.exists():
            annotation_csv = open(annotation_file, 'w+', newline='')
            annotation_writer = csv.writer(annotation_csv)
            annotation_writer.writerow(['id','World','c_x','c_y','Sensor','time(ex. 13.5 = 1:30 pm)','overcast [0:1]','rain [0:1]','fog [0:1]','orig_file'])
            annotation_csv.close()
            print("SearchAndMove_Empty::InitialisedAnnotationFile")
            
        # Open to append data
        annotation_csv = open(annotation_file, 'a', newline='')
        annotation_writer = csv.writer(annotation_csv)
        data = [ID,world,c_x,c_y,'Vis',time,overcast,rain,fog,file_name]; print(data)
        annotation_writer.writerow(data)
        annotation_csv.close()
        print("SearchAndMove_Empty::Annotation::"+str(data))

        dest_dir_path = dest_dir_path / Path(f'%06d.png'%(ID))
        move_file(filepath, dest_dir_path)
        print("AnnotateAndMove_Full::Moved::"+str(filepath))

def AnnotateAndMove_Full(screenshot_path,dest_dir,copies=4):

    # Check if the last file of the set is has been ganarated
    for fullSet_lastImage in glob.glob(str(screenshot_path)+f'/*%d-with.png'%copies):
        if("Extract information from filename"):
            file_name = fullSet_lastImage.split('\\')[-1]
            print("Parsing :",file_name)
            prefix = ('-'.join(file_name.split('-')[:-2]))
            filename_tokens = file_name.split('-')
            idx             = 0
            world           = filename_tokens[idx]; idx+=1
            v_name          = filename_tokens[idx]; idx+=1
            v_x,v_y         = [float(coo) for coo in filename_tokens[idx][1:-1].split(',') ]; idx+=1
            c_x,c_y         = [float(coo) for coo in filename_tokens[idx][1:-1].split(',') ]; idx+=1
            time            = int(filename_tokens[idx]); idx+=1
            overcast        = float(filename_tokens[idx]); idx+=1
            rain            = float(filename_tokens[idx]); idx+=1
            fog             = float(filename_tokens[idx]); idx+=1
            if(v_name not in ARMA3_DG_vehiclesClasses.keys()):
                v_class = 'NA'
            else:
                v_class = ARMA3_DG_vehiclesClasses[v_name]
        
        input("Press Enter to continue...")
        
        if("Check all files exist"):
            print("AnnotateAndMove_Full::cheking that full set exists.",end='::')
            valid_paths_with = []
            valid_paths_wout = []
            set_exists = True
            for copy in range(copies):
                idx = copy+1
                check_path_with = screenshot_path / Path(prefix+f'-%d-with.png'%idx)
                check_path_wout = screenshot_path / Path(prefix+f'-%d-wout.png'%idx)
                if (not check_path_with.exists() or not check_path_with.exists()):
                    #print(str(check_path_with) + ' AND ' + str(check_path_wout) + "  NOK")
                    set_exists = False
                    break
                else:
                    valid_paths_with.append(check_path_with)
                    valid_paths_wout.append(check_path_wout)
                    #print(str(check_path_with) + ' AND ' + str(check_path_wout) + "  OK")
            if(not set_exists):
                print("Incomplete Set::Breaking")
            else:
                print("Complete Set")

        if("Get Bounding box for images"):
            if(set_exists):
                BB = get_BB_BackroundSubtraction(valid_paths_wout,valid_paths_with,inclussionDiameter=400,open_mask=True,showIntermediate=True)
            objectFound = (BB is not None)
            if(objectFound):
                print("AnnotateAndMove_Full::Object located::",BB)
                BB = np.array(BB)+[-7,-7,14,14] # Scale up the BB to encaptulate more
                TL = (BB[0], BB[1])
                TR = (BB[0] + BB[2], BB[1])
                BL = (BB[0], BB[1] + BB[3])
                BR = (BB[0] + BB[2], BB[1] + BB[3])
                marked = cv2.rectangle(cv2.imread(valid_paths_with[-1],1), TL, BR, (255, 0, 0), 2)
            else:
                print("AnnotateAndMove_Full::Object not located")
                TL=TR=BL=BR=(-1,-1)
            
        if("Get appropriate world directory"):
            # Move to world directory
            if objectFound:
                dest_dir_path = dest_dir / Path(world+'_object')
            else:
                dest_dir_path = dest_dir / Path(world+'_manualLabeling')
            if not Path.exists(dest_dir_path):
                os.makedirs(str(dest_dir_path), 0o777)  # create the directory if it doesnt exist
            print("AnnotateAndMove_Full::DestinationDir::"+str(dest_dir_path))
            
        if("Produce new ID"):
            # Get Ascending ID
            ID = get_next_id(dest_dir_path)
            print("AnnotateAndMove_Full::NewID::"+str(ID))

        if("Initialize Annotation file if required"):
            annotation_file = dest_dir_path / Path('AnnotationData.csv')
            if not annotation_file.exists():
                annotation_csv = open(annotation_file, 'w+', newline='')
                annotation_writer = csv.writer(annotation_csv)
                annotation_writer.writerow(['id','World','TL','TR','BR','BL','v_name','v_class','v_x','v_y','c_x','c_y','Sensor','time(ex. 13.5 = 1:30 pm)','overcast [0:1]','rain [0:1]','fog [0:1]','orig_file'])
                annotation_csv.close()
                print("AnnotateAndMove_Full::InitialisedAnnotationFile")

        if("Append annoatation data"):
            annotation_csv = open(annotation_file, 'a', newline='')
            annotation_writer = csv.writer(annotation_csv)
            
            if(objectFound):
                data = [ID,world,TL,TR,BR,BL,v_name,v_class,v_x,v_y,c_x,c_y,'Vis',time,overcast,rain,fog,file_name];
            annotation_writer.writerow(data)
            annotation_csv.close()
            print("AnnotateAndMove_Full::Annotation::"+str(data))

        if("Now move files to appropriate folder"):
            if(objectFound):
                #move one blank(PNG), one with the object(PNG) and the marked resized(JPG)
                # With Object
                dest_path = dest_dir_path / Path(f'%06d.png'%(ID))
                move_file(valid_paths_with[-1], dest_path)
                print("AnnotateAndMove_Full::Moved::"+str(valid_paths_with[-1]))
                # Blank
                dest_path = dest_dir_path / Path(f'%06d_blank.png'%(ID))
                move_file(valid_paths_wout[-1], dest_path)
                print("AnnotateAndMove_Full::Moved::"+str(valid_paths_wout[-1]))
                # Marked
                dest_path = dest_dir_path / Path(f'%06d_marked.jpg'%(ID))
                cv2.imwrite(dest_path,marked)
                print("AnnotateAndMove_Full::Stored::"+str(dest_path))
                # Delete the rest
                for srcpath in valid_paths_wout[:-1]:
                    os.remove(srcpath)
                    print("AnnotateAndMove_Full::Removed::"+str(srcpath))
                for srcpath in valid_paths_with[:-1]:
                    os.remove(srcpath)
                    print("AnnotateAndMove_Full::Removed::"+str(srcpath))
            else:
                for srcpath in valid_paths_wout:
                    file_name = fullSet_lastImage.split('\\')[-1]
                    dest_path = dest_dir_path / Path(file_name)
                    move_file(srcpath, dest_path)
                    print("AnnotateAndMove_Full::Moved::"+str(srcpath))
                for srcpath in valid_paths_with:
                    file_name = fullSet_lastImage.split('\\')[-1]
                    dest_path = dest_dir_path / Path(file_name)
                    move_file(srcpath, dest_path)
                    print("AnnotateAndMove_Full::Moved::"+str(srcpath))



SearchAndMove_Empty(screenshot_path,dest_dir)
AnnotateAndMove_Full(screenshot_path,dest_dir)

SearchAndMove_Empty::ParsingMalden-[10.6,1.4]-10-0.1-0.2-0.3-empty.png
SearchAndMove_Empty::DestinationDirD:\synthetic_dataset\arma\Malden_empty
[18, 'Malden', 10.6, 1.4, 'Vis', 10, 0.1, 0.2, 0.3, 'Malden-[10.6,1.4]-10-0.1-0.2-0.3-empty.png']
SearchAndMove_Empty::Annotation::[18, 'Malden', 10.6, 1.4, 'Vis', 10, 0.1, 0.2, 0.3, 'Malden-[10.6,1.4]-10-0.1-0.2-0.3-empty.png']
AnnotateAndMove_Full::Moved::C:\Users\user1\Documents\Arma 3\Screenshots\Malden-[10.6,1.4]-10-0.1-0.2-0.3-empty.png
SearchAndMove_Empty::ParsingMalden-[6052.93,1.4]-10-0.1-0.2-0.3-empty.png
SearchAndMove_Empty::DestinationDirD:\synthetic_dataset\arma\Malden_empty
[19, 'Malden', 6052.93, 1.4, 'Vis', 10, 0.1, 0.2, 0.3, 'Malden-[6052.93,1.4]-10-0.1-0.2-0.3-empty.png']
SearchAndMove_Empty::Annotation::[19, 'Malden', 6052.93, 1.4, 'Vis', 10, 0.1, 0.2, 0.3, 'Malden-[6052.93,1.4]-10-0.1-0.2-0.3-empty.png']
AnnotateAndMove_Full::Moved::C:\Users\user1\Documents\Arma 3\Screenshots\Malden-[6052.93,1.4]-10-0.1-0.2-0.3-empty.png


In [8]:
screenshot_path = Path(os.path.expanduser('~\\Documents\\Arma 3\\Screenshots'))
dest_dir        = Path(os.path.expanduser('D:\\synthetic_dataset\\arma'))

copies = 4
# Check if the last file of the set is has been ganarated
for fullSet_lastImage in glob.glob(str(screenshot_path)+f'/*%d-with.png'%copies):
    if("Extract information from filename"):
        file_name = fullSet_lastImage.split('\\')[-1]
        print("Parsing :",file_name)
        prefix = ('-'.join(file_name.split('-')[:-2]))
        filename_tokens = file_name.split('-')
        idx             = 0
        world           = filename_tokens[idx]; idx+=1
        v_name          = filename_tokens[idx]; idx+=1
        v_x,v_y         = [float(coo) for coo in filename_tokens[idx][1:-1].split(',') ]; idx+=1
        c_x,c_y         = [float(coo) for coo in filename_tokens[idx][1:-1].split(',') ]; idx+=1
        time            = int(filename_tokens[idx]); idx+=1
        overcast        = float(filename_tokens[idx]); idx+=1
        rain            = float(filename_tokens[idx]); idx+=1
        fog             = float(filename_tokens[idx]); idx+=1
        if(v_name not in vehiclesClass.keys()):
            v_class = 'NA'
        else:
            v_class = vehiclesClass[v_name]
    
    input("Press Enter to continue...")
    
    if("Check all files exist"):
        print("AnnotateAndMove_Full::cheking that full set exists.")
        valid_paths_with = []
        valid_paths_wout = []
        set_exists = True
        for copy in range(copies):
            idx = copy+1
            check_path_with = screenshot_path / Path(prefix+f'-%d-with.png'%idx)
            check_path_wout = screenshot_path / Path(prefix+f'-%d-wout.png'%idx)
            if (not check_path_with.exists() or not check_path_with.exists()):
                print(str(check_path_with) + ' AND ' + str(check_path_wout) + "  NOK")
                set_exists = False
                break
            else:
                valid_paths_with.append(check_path_with)
                valid_paths_wout.append(check_path_wout)
                print(str(check_path_with) + ' AND ' + str(check_path_wout) + "  OK")

    if("Get Bounding box for images"):
        if(set_exists):
            BB = get_BB_BackroundSubtraction(valid_paths_wout,valid_paths_with,inclussionDiameter=400,open_mask=True,showIntermediate=True)
        objectFound = (BB is not None)
        if(objectFound):
            print("AnnotateAndMove_Full::Object located::",BB)
            BB = np.array(BB)+[-7,-7,14,14] # Scale up the BB to encaptulate more
            TL = (BB[0], BB[1])
            TR = (BB[0] + BB[2], BB[1])
            BL = (BB[0], BB[1] + BB[3])
            BR = (BB[0] + BB[2], BB[1] + BB[3])
            marked = cv2.rectangle(cv2.imread(valid_paths_with[-1],1), TL, BR, (255, 0, 0), 2)
        else:
            print("AnnotateAndMove_Full::Object not located")
            TL=TR=BL=BR=(-1,-1)
        
    if("Get appropriate world directory"):
        # Move to world directory
        if objectFound:
            dest_dir_path = dest_dir / Path(world+'_object')
        else:
            dest_dir_path = dest_dir / Path(world+'_manualLabeling')
        if not Path.exists(dest_dir_path):
            os.makedirs(str(dest_dir_path), 0o777)  # create the directory if it doesnt exist
        print("AnnotateAndMove_Full::DestinationDir::"+str(dest_dir_path))
        
    if("Produce new ID"):
        # Get Ascending ID
        ID = get_next_id(dest_dir_path)
        print("AnnotateAndMove_Full::NewID::"+str(ID))

    if("Initialize Annotation file if required"):
        annotation_file = dest_dir_path / Path('AnnotationData.csv')
        if not annotation_file.exists():
            annotation_csv = open(annotation_file, 'w+', newline='')
            annotation_writer = csv.writer(annotation_csv)
            annotation_writer.writerow(['id','World','TL','TR','BR','BL','v_name','v_class','v_x','v_y','c_x','c_y','Sensor','time(ex. 13.5 = 1:30 pm)','overcast [0:1]','rain [0:1]','fog [0:1]','orig_file'])
            annotation_csv.close()
        print("AnnotateAndMove_Full::InitialisedAnnotationFile")

    if("Append annoatation data"):
        annotation_csv = open(annotation_file, 'a', newline='')
        annotation_writer = csv.writer(annotation_csv)
        
        if(objectFound):
            data = [ID,world,TL,TR,BR,BL,v_name,v_class,v_x,v_y,c_x,c_y,'Vis',time,overcast,rain,fog,file_name];
        annotation_writer.writerow(data)
        annotation_csv.close()
        print("AnnotateAndMove_Full::Annotation::"+str(data))

    if("Now move files to appropriate folder"):
        if(objectFound):
            #move one blank(PNG), one with the object(PNG) and the marked resized(JPG)
            # With Object
            dest_path = dest_dir_path / Path(f'%06d.png'%(ID))
            move_file(valid_paths_with[-1], dest_path)
            print("AnnotateAndMove_Full::Moved::"+str(valid_paths_with[-1]))
            # Blank
            dest_path = dest_dir_path / Path(f'%06d_blank.png'%(ID))
            move_file(valid_paths_wout[-1], dest_path)
            print("AnnotateAndMove_Full::Moved::"+str(valid_paths_wout[-1]))
            # Marked
            dest_path = dest_dir_path / Path(f'%06d_marked.jpg'%(ID))
            cv2.imwrite(dest_path,marked)
            print("AnnotateAndMove_Full::Stored::"+str(dest_path))
            # Delete the rest
            for srcpath in valid_paths_wout[:-1]:
                os.remove(srcpath)
                print("AnnotateAndMove_Full::Removed::"+str(srcpath))
            for srcpath in valid_paths_with[:-1]:
                os.remove(srcpath)
                print("AnnotateAndMove_Full::Removed::"+str(srcpath))
        else:
            for srcpath in valid_paths_wout:
                file_name = fullSet_lastImage.split('\\')[-1]
                dest_path = dest_dir_path / Path(file_name)
                move_file(srcpath, dest_path)
                print("AnnotateAndMove_Full::Moved::"+str(srcpath))
            for srcpath in valid_paths_with:
                file_name = fullSet_lastImage.split('\\')[-1]
                dest_path = dest_dir_path / Path(file_name)
                move_file(srcpath, dest_path)
                print("AnnotateAndMove_Full::Moved::"+str(srcpath))


Parsing : Malden-Leopard2A6HEL_2-[6748,6312]-[6506.16,6161.99]-16-0-0-0-4-with.png
AnnotateAndMove_Full::cheking that full set exists.
C:\Users\user1\Documents\Arma 3\Screenshots\Malden-Leopard2A6HEL_2-[6748,6312]-[6506.16,6161.99]-16-0-0-0-1-with.png AND C:\Users\user1\Documents\Arma 3\Screenshots\Malden-Leopard2A6HEL_2-[6748,6312]-[6506.16,6161.99]-16-0-0-0-1-wout.png  OK
C:\Users\user1\Documents\Arma 3\Screenshots\Malden-Leopard2A6HEL_2-[6748,6312]-[6506.16,6161.99]-16-0-0-0-2-with.png AND C:\Users\user1\Documents\Arma 3\Screenshots\Malden-Leopard2A6HEL_2-[6748,6312]-[6506.16,6161.99]-16-0-0-0-2-wout.png  OK
C:\Users\user1\Documents\Arma 3\Screenshots\Malden-Leopard2A6HEL_2-[6748,6312]-[6506.16,6161.99]-16-0-0-0-3-with.png AND C:\Users\user1\Documents\Arma 3\Screenshots\Malden-Leopard2A6HEL_2-[6748,6312]-[6506.16,6161.99]-16-0-0-0-3-wout.png  OK
C:\Users\user1\Documents\Arma 3\Screenshots\Malden-Leopard2A6HEL_2-[6748,6312]-[6506.16,6161.99]-16-0-0-0-4-with.png AND C:\Users\user1\Doc

In [10]:
if(True):print("fsdf")

fsdf


In [20]:
screenshot_path = Path(os.path.expanduser('~\\Documents\\Arma 3\\Screenshots'))
dest_dir        = Path(os.path.expanduser('D:\\synthetic_dataset\\arma'))

SearchAndMove_Empty(screenshot_path,dest_dir)

[13, 'Malden', 6278.37, 6480.72, 'Vis', 12, 0.0, 0.0, 0.0, 'Malden-[6278.37,6480.72]-12-0-0-0-empty.png']
[14, 'Malden', 6523.14, 5978.8, 'Vis', 10, 0.0, 0.0, 0.0, 'Malden-[6523.14,5978.8]-10-0-0-0-empty.png']
[15, 'Malden', 6572.79, 6506.52, 'Vis', 11, 0.0, 0.0, 0.0, 'Malden-[6572.79,6506.52]-11-0-0-0-empty.png']
[16, 'Malden', 6618.26, 6033.73, 'Vis', 8, 0.1, 0.5, 0.1, 'Malden-[6618.26,6033.73]-8-0.1-0.5-0.1-empty.png']
[17, 'Malden', 6658.2, 6039.43, 'Vis', 11, 0.0, 0.0, 0.0, 'Malden-[6658.2,6039.43]-11-0-0-0-empty.png']


In [7]:

screenshot_path = Path(os.path.expanduser('~\\Documents\\Arma 3\\Screenshots'))
dest_dir        = Path(os.path.expanduser('D:\\synthetic_dataset\\arma'))

for filepath in glob.glob(str(screenshot_path)+'/*empty.png'):
    file_name = filepath.split('\\')[-1]
    print(file_name)
    filename_tokens = file_name.split('-')
    world = filename_tokens[0]
    c_x,c_y = [float(coo) for coo in filename_tokens[1][1:-1].split(',') ]
    print(c_x,c_y)

Malden-[6052.93,1.4]-empty.png
6052.93 1.4
Malden-[6132.66,2.3]-empty.png
6132.66 2.3


In [3]:
screenshot_path = Path(os.path.expanduser('~\\Documents\\Arma 3\\Screenshots'))
dest_dir = Path(os.path.expanduser('D:\\synthetic_dataset\\arma'))

if not Path.exists(dest_dir):
    os.makedirs(str(dest_dir), 0o777)  # create the directory if it doesnt exist
# if not screenshot_path.exists(): # I will fix this later
    # need to do some error stuff here
# open annotation csv file and write the header data. THIS WILL BLOW AWAY THE FILE CONTENTS
annotation_file_path = Path(str(dest_dir) + '\\AnnotationData.csv')
annotation_csv = open(annotation_file_path, 'w+', newline='')
annotation_writer = csv.writer(annotation_csv)
annotation_writer.writerow(['filename','vehiclename','target_x','target_y','target_z','camera_x','camera_y','camera_z',
                            'foglevel','time','TR','BR','TL','BL'])
annotation_csv.close()

In [ ]:

def image_file_mover(check_delay, config_params, cam_pos_list):
    """ Parameters:
            check_delay: how long to wait between checking for image file existence
            config_params: a dictionary of relevant configuration parameters used in this run of the program
            cam_pos_list: the list of generated camera positions
        Description:
        This function will check a pre-determined directory for pre-determined file name and will continue to check,
    sleeping for <check_delay> seconds between each check. If the file exists, it will move it to a separate
    pre-determined directory. check_delay should be 1/2 the time it takes to rotate the camera to the next position.
        Future work could be to set the process affinity here to a different core, but isn't important right now.
    """

    # relative counters for tracking occurrences
    world_pos_idx = 0
    vehicle_idx = 0
    fog_ct = 0
    tod_ct = 0
    veh_orient_ct = 0  # number of vehicle orientations seen
    cam_pos_ct = 0

    screenshot_path = Path(os.path.expanduser('~\\Documents\\Arma 3\\Screenshots'))
    dest_dir = Path(os.path.expanduser('D:\\synthetic_dataset\\arma'))
    
    if not Path.exists(dest_dir):
        os.makedirs(str(dest_dir), 0o777)  # create the directory if it doesnt exist
    # if not screenshot_path.exists(): # I will fix this later
        # need to do some error stuff here
    # open annotation csv file and write the header data. THIS WILL BLOW AWAY THE FILE CONTENTS
    annotation_file_path = Path(str(dest_dir) + '\\AnnotationData.csv')
    annotation_csv = open(annotation_file_path, 'w+', newline='')
    annotation_writer = csv.writer(annotation_csv)
    annotation_writer.writerow(['filename','vehiclename','v_x','v_y','c_x','c_y','overcast','rain','fog','TR','BR','TL','BL'])
    annotation_csv.close()
    # re-open file for append to write the data. I will leave this file open for time concerns while moving files
    annotation_csv = open(annotation_file_path, 'a', newline='')
    annotation_writer = csv.writer(annotation_csv)

    # while img_ctr <= num_images and blank_ctr <= num_images:
    while img_ctr <= num_images:
        screenshot = Path(str(screenshot_path) + '\\arma3screenshot.png')
        blank_screenshot = Path(str(screenshot_path) + '\\arma3blank.png')
        # check for data screenshot
        if screenshot.exists() and not screenshot.is_dir():
            # exit with error code 4 if this fails...
            dest = Path(os.path.expanduser('~\\Documents\\SyntheticDataGen\\' + str(img_ctr) + '.png'))
            move_file(screenshot, dest)

            # append this file's metadata to the annotation csv file here
            filename = '%s.png' % img_ctr
            time_frac, time_hr = math.modf(6 + tod_ct * config_params['time_incr'])
            time_min = int(round(time_frac * 60))
            timestr = '%d:%d' % (time_hr, time_min)
            data = [filename,
                    config_params['vehicle_names'][vehicle_idx],
                    config_params['world_positions'][world_pos_idx][0],
                    config_params['world_positions'][world_pos_idx][1],
                    config_params['world_positions'][world_pos_idx][2],
                    cam_pos_list[cam_pos_ct][0],
                    cam_pos_list[cam_pos_ct][1],
                    cam_pos_list[cam_pos_ct][2],
                    fog_ct * config_params['fog_incr'],
                    timestr]
            annotation_writer.writerow(data)
            cam_pos_ct = cam_pos_ct + 1
            # calculate where in each environmental effect the next image will be
            if cam_pos_ct == num_cam_positions:
                cam_pos_ct = 0
                veh_orient_ct = veh_orient_ct + 1
                if veh_orient_ct == num_vehicle_rotations:
                    veh_orient_ct = 0
                    fog_ct = fog_ct + 1
                    if fog_ct == num_fog_settings:
                        fog_ct = 0
                        tod_ct = tod_ct + 1
                        if tod_ct == num_time_settings:
                            tod_ct = 0
                            world_pos_idx = world_pos_idx + 1
                            if world_pos_idx == num_world_positions:
                                world_pos_idx = 0
                                vehicle_idx = vehicle_idx + 1

            img_ctr += 1  # increment the image counter after a successful move
            # I have never seen it print these
            if vehicle_idx == num_vehicles and img_ctr <= num_images:
                print("There appears to be an error counting images...")
                print("Current number of vehicles evaluated: %d" % vehicle_idx)
                print("Current number of images seen: %d / %d" % (img_ctr, num_images))

        # if data screenshot is not present, check for img comparison screenshot
        elif blank_screenshot.exists() and not blank_screenshot.is_dir():
            dest = Path(os.path.expanduser('~\\Documents\\SyntheticDataGen\\' + str(blank_ctr) + '_blank.png'))
            move_file(blank_screenshot, dest)
            blank_ctr += 1  # increment the image counter after a successful move

        time.sleep(check_delay)  # sleep for check_delay seconds so we don't hammer the cpu unnecessarily

    annotation_csv.close()
